This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'API_KEY'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
zeiss_samp = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2020-01-05&end_date=2020-01-05&api_key=API_KEY')


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(zeiss_samp.json())

{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (2020-01-09) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2020-01-23T22:16:41.432Z', 'newest_available_date': '2020-01-23', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': None, 'transform': None, 'column_index': None, 'start_date': '2020-01-05', 'end_date': '2020-01-05', 'data': [], 'collapse': None, 'order': None, 'database_id': 6129}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

## 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [5]:
# Get data
zeiss_17 = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=API_KEY')


##  2. Convert the returned JSON object into a Python dictionary.

In [6]:
# Convert it to the dict
z17_js = zeiss_17.json()

## Data Extraction

In [7]:
# Now let's initialize important variables
zeiss_open = []
zeiss_high = []
zeiss_low = []
zeiss_close = []
zeiss_change = [0] 
zeiss_daily_change = []
zeiss_volume = []

In [8]:
for i in range(len(z17_js['dataset_data']['data'])):
    dummy_open = z17_js['dataset_data']['data'][i][1]
    dummy_close = z17_js['dataset_data']['data'][i][4]
    dummy_high = z17_js['dataset_data']['data'][i][2]
    dummy_low = z17_js['dataset_data']['data'][i][3]
    dummy_vol = z17_js['dataset_data']['data'][i][6]
    if isinstance(dummy_open,float):
        zeiss_open.append(dummy_open)
    if isinstance(dummy_close,float):
        zeiss_close.append(dummy_close)
    if isinstance(dummy_high,float) and isinstance(dummy_low,float):    
        zeiss_daily_change.append(dummy_high - dummy_low)
    if i != 0:
        zeiss_change.append(zeiss_close[i] - zeiss_close[i-1])
    if isinstance(dummy_vol,float):
        zeiss_volume.append(dummy_vol)

## 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [9]:
print('Highest opening price is: $%4.2f'% max(zeiss_open))
print('Lowest opening price is: $%4.2f'% min(zeiss_open))

Highest opening price is: $53.11
Lowest opening price is: $34.00


## 4. What was the largest change in any one day (based on High and Low price)?

In [10]:
print('The largest change in any one day is: $%4.2f'% max(zeiss_daily_change))

The largest change in any one day is: $2.81


## 5. What was the largest change between any two days (based on Closing Price)?

In [11]:
print('The largest change in between 2 days based on closing price is: $%4.2f'% max(zeiss_change))

The largest change in between 2 days based on closing price is: $2.56


## 6. What was the average daily trading volume during this year?

In [12]:
average_vol = sum(zeiss_volume)/len(zeiss_volume)
print('The average traded volume is: $%4.2f'% average_vol)

The average traded volume is: $89124.34


## 7.What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [13]:
median = sorted(zeiss_volume)[int(len(zeiss_volume)/2)]
print('Median volume is: $%4.2f'% median)

Median volume is: $76286.00
